In [1]:
%load_ext autoreload
%autoreload 2

from copy import copy, deepcopy
import logging
import configparser
from sqlalchemy import MetaData, create_engine
import itertools
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
import pgdb
import s3fs
import boto3
import seaborn as sns 
import matplotlib.pyplot as plt
from datetime import date as dt
from datetime import datetime, timedelta
import datetime
from LR_prod import *

In [2]:
class Query_Redshit: 
    def __init__(self,dbname): 
        self.dbname = dbname
        self.conn = None

    def _get_credentials(self): 
        database = self.dbname

        if database == 'vdp':
        # ingress = {
        #     'securityGroupId': 'sg-1e2e7460',  # redshift security group
        #     'port': 8192,  # redshift port
        #     'region': 'us-west-2',  # redshift region
        #     'materialSet': 'com.amazon.access.Kindle-Planning-Prod-EiderScience-1'  # iam user able to add ingress rule
        # }
        # eider.network.authorizeIngress(**ingress)
        # vdp_credentials = eider.odin.getCredential("com.amazon.kindle-planning-redshift.ro.prod.eider")
            connection = {
                'host': 'kindle-planning-redshift.cfbl7dhwcatl.us-west-2.redshift.amazonaws.com',
                'dbname': 'prod',
                'user': 'kindle_planning_ro', 
                'password': 'cYsYP6Mhd#VFec%6TrBA',
                'port':8192
            }
        elif database == 'dmap':
            # ingress = {
            #     'securityGroupId': 'sg-0154da99c0fd65b92',  # redshift security group
            #     'port': 8192,  # redshift port
            #     'region': 'us-east-1',  # redshift region
            #     'materialSet': 'com.amazon.access.Dial-DMaP-Data-Prod-EiderScienceTeam-1'  # iam user able to add ingress rule
            # }
            # eider.network.authorizeIngress(**ingress)
            # credentials = eider.odin.getCredential("com.amazon.dmap.data.redshift.prod.ro_direct")
            connection = {
                'host': 'dmap-data-cluster-prod.cbdo75nipkyk.us-east-1.redshift.amazonaws.com',
                'dbname': 'dmapdataclusterprod',
                'user':'dmap_data_prod_rw',
                'password': '%TGY2+)1BU2daHs' ,
                'port':8192
            }
            # conn = pgdb.connect(**dmap_connection)
            # eider.network.revokeIngress(**ingress)
        # Analytics Cluster    
        elif database == 'analytics':
            # ingress = {
            #     'securityGroupId': 'sg-0a8c44f0f1f6936f9',  # redshift security group
            #     'port': 8192,  # redshift port
            #     'region': 'us-east-1',  # redshift region
            #     'materialSet': 'com.amazon.access.Kindle-Planning-Corp-EiderDmap-1'  # iam user able to add ingress rule
            # }
            # eider.network.authorizeIngress(**ingress)
            # analytic_credentials = eider.odin.getCredential("com.amazon.kindle-planning-redshift.device-planning-analytics.dial_dmap_ro")
            connection = {
                'host': 'device-planning-analytics.clszfooyvile.us-east-1.redshift.amazonaws.com',
                'dbname': 'deviceplanning',
                'user': 'dial_dmap_ro', 
                'password': 'RO85711f12260729afbfdfaf2ddaeb139f',
                'port':8192
            }
            # conn = pgdb.connect(**analytic_connection)
            # eider.network.revokeIngress(**ingress)
        # DMAP Beta Cluster    
        elif database == 'dmap_beta':
            # ingress = {
            #     'securityGroupId': 'sg-072276cf843295057',
            #     'port': 8192,
            #     'region': 'us-east-1',
            #     'materialSet': 'com.amazon.access.Dial-DMaP-Data-Beta-S3user-1'
            # }
            # eider.network.authorizeIngress(**ingress)
            
            # credentials = eider.odin.getCredential("com.amazon.dmap.data.redshift.beta.dmap_beta_rw")
            connection = {
                'host': 'dmap-data-cluster-beta.c9aj4xzdfqrn.us-east-1.redshift.amazonaws.com',
                'dbname': 'dmapdataclusterbeta',
                'user':'eider_rw',
                'password': '3NCPSnM06smf9Q0mFw7yRSwhwccOYyflvGJPgXaM',
                'port':8192
            }
            # conn = pgdb.connect(**dmap_connection)
            # eider.network.revokeIngress(**ingress)
        # VDP Science Prod Cluster
        elif database == 'vdp_science':
            # ingress = {
            #     'securityGroupId': 'sg-056373d3b84cc25ae',
            #     'port': 8192,
            #     'region': 'us-east-1',
            #     'materialSet': 'com.amazon.access.Dial-Vdp-Science-Data-Catalog-Prod-Eider-VDP-User-1'
            # }
            # eider.network.authorizeIngress(**ingress)
            
            # credentials = eider.odin.getCredential("com.amazon.dial-vdp-science.vdp-redshift.vdpscience_rs_prod")
            connection = {
                'host': 'vdpscienceprod.ceuabgn11vdw.us-east-1.redshift.amazonaws.com',
                'dbname': 'vdpscienceprod',
                'user': 'vdpscience_rs_prod',
                'password': 'B9ZscC3y$WSXvxb',
                'port':8192
            }
            # conn = pgdb.connect(**vdp_connection)
        # eider.network.revokeIngress(**ingress)
   
        else:
            print("Please choose database from ('vdp','dmap','analytics','dmap_beta','vdp_science')")
            return
        self.credentials = connection


    def connect(self): 
        # print("connecting {}".format(self.dbname))
        self._get_credentials()
        self.conn = pgdb.connect(**self.credentials)

    def query(self, query_string): 
        # print(query_string)
        if self.conn is None: # connect when self.conn is None
            self.connect()
        df = pd.read_sql(sql = query_string, con = self.conn)
        self.conn.close()
        return df 
    

In [3]:
def get_marketplace(country):
    marketplace_mapping = {'US': ['1', '1338980', '1071830', '1259700'],
    'UK': ['3', '330551'],
    'DE': ['4', '330871'],
    'CA': ['7', '1398340'],
    'JP': ['6', '121322'],
    'IN': ['44571'],
    'AU': ['111172'],
    'FR': ['5', '330921'],
    'IT': ['35691', '330711'],
    'ES': ['44551', '330731'],
    'MX': ['771770'],
    'BR': ['526970'], 
    'CN': ['NA']}
    marketplace_id = marketplace_mapping[country]
    marketplace_id = ','.join(["'"+x+"'" for x in marketplace_id])
    marketplace_id = '(' + marketplace_id + ')'
    return marketplace_id

def query_actuals(country, channel, device_type, start_date, end_date):

    query = """
    select date, country, channel, device_type, program, a.asin
    , sum(actual) as actual
    from attainments_cache a join asins using(asin)
    where country = '{country}'
    and channel = '{channel}'
    and device_type = '{device_type}'
    and date between '{start_date}' and '{end_date}'
    and program not in ('Doppler','Fox') 
    group by 1,2,3,4,5,6
    order by 1,2,3,4,5,6""".format(country = country, channel = channel, device_type = device_type,
                                  start_date = start_date, end_date = end_date)
    vdp_db = Query_Redshit('vdp')
    df = vdp_db.query(query)
    df.date = pd.to_datetime(df.date)

    return df

def query_vdps(country, channel, device_type, start_date, end_date, fct_version):

    query = """select date, from_country as country, channel, device_type, program, asin, sum(vdp_plus_units) as vdp_plus_units
    from vdp_retailer_vdps
    where from_country = '{country}'
    and channel = '{channel}'
    and device_type = '{device_type}'
    and date between '{start_date}' and '{end_date}'
    and fct_version = '{fct_version}'
    and program not in ('Doppler','Fox') 
    group by 1,2,3,4,5,6""".format(country = country, channel = channel, device_type = device_type
                                , start_date = start_date, end_date = end_date, fct_version = fct_version)
    df = query_redshift(query)
    df.date = pd.to_datetime(df.date)

    return df


def query_asin_end(country, channel, device_type):
    query = """
    select program, asin, min(date) as asin_start_date, max(date) as asin_end_date
    from ATTAINMENTS_CACHE join asins using(asin)
    where 1=1
    and country = '{country}'
    and channel = '{channel}'
    and device_type = '{device_type}'
    and baseline_units > 0
    group by 1,2
    order by 1""".format(country = country, channel = channel, device_type = device_type)
    vdp_science_db = Query_Redshit('vdp')
    df = vdp_science_db.query(query)
    df.asin_end_date = pd.to_datetime(df.asin_end_date)
    df.asin_start_date = pd.to_datetime(df.asin_start_date)
    return df

def query_asp(country, channel, device_type, start_date, end_date):

    query = """
    select order_day as date, country, channel, a.device_type, program, a.asin
    , alp
    , asp
    , ordered_qty as qty
    from adept.price_elasticity_by_saket_asin a join adept.asins using(asin)
    where country = '{country}'
    and a.channel = '{channel}'
    and a.device_type = '{device_type}'
    and a.order_day between '{start_date}' and '{end_date}'""".format(country = country, channel = channel
    , device_type = device_type, start_date = start_date, end_date = end_date)
    vdp_science_db = Query_Redshit('vdp_science')
    df = vdp_science_db.query(query)
    df.date = pd.to_datetime(df.date)

    return df

def query_vlt(country, channel , device_type, start_date, end_date):
    """
        query vlt
    """
    market_place_id = get_marketplace(country)
    query = """
    select  date(snapshot_day) date, asin, avg(vlt) as vlt
    from dialvdpfeaturecatalog.d_daily_asin_vlt 
    where marketplace_id in {market_place_id}
    group by 1,2
    order by date""".format(market_place_id = market_place_id)
    vdp_science_prod_db = Query_Redshit('vdp_science')
    df = vdp_science_prod_db.query(query)
    df.date = pd.to_datetime(df.date)
    return df

def query_asin_attr(country, channel, device_type, start_date, end_date):
    """
        query asin attributes
    """
    query = "select *  from dialvdpfeaturecatalog.d_device_asins \
    where country = '{country}' \
    and device_type = '{device_type}'".format(country = country, device_type = device_type)

    vdp_science_prod_db = Query_Redshit('vdp_science')
    df = vdp_science_prod_db.query(query)
    if 'date' in df.columns:
        df.date = pd.to_datetime(df.date)
    return df

def query_holidays(country, channel, device_type, holiday_name, start_year, end_year):

    query = """
    select date(calendar_date)
    from dialvdpfeaturecatalog.d_holiday_attributes 
    WHERE  country = '{country}' 
           AND calendar_year BETWEEN '{start_year}' AND '{end_year}' 
           AND holiday = '{holiday_name}'
    """.format(country=country,
                start_year=start_year,
                end_year=end_year,
                holiday_name=holiday_name)
    vdp_science_prod_db = Query_Redshit('vdp_science')
    df = vdp_science_prod_db.query(query)
    dates = df.date.tolist()
    return dates

def generate_holidays(country, channel, device_type, start_date, end_date):


    index = pd.date_range(start=start_date, end=end_date, freq='D')
    df_holiday_binary = pd.DataFrame(index=index, columns=['date'])
    df_holiday_binary.date = df_holiday_binary.index
    start_year = pd.to_datetime(start_date).year 
    end_year = pd.to_datetime(end_date).year

    dates_Prime = query_holidays(country,channel,device_type,'Prime Day',start_year,end_year)
    dates_Prime += [dt(2018, 7, 16)]


    dates_Prime_m1w = [date for sublist in
                           [pd.date_range(x - timedelta(7), x - timedelta(2)).tolist()
                            for x in dates_Prime]
                           for date in sublist]
    dates_Prime_m2w = [date for sublist in
                       [pd.date_range(x - timedelta(14), x - timedelta(8)).tolist()
                        for x in dates_Prime]
                       for date in sublist]
    dates_Prime_p1w = [date for sublist in
                       [pd.date_range(x + timedelta(2), x + timedelta(7)).tolist()
                        for x in dates_Prime]
                       for date in sublist]
    dates_Prime_m3w = [date for sublist in
                       [pd.date_range(x - timedelta(21), x - timedelta(15)).tolist()
                        for x in dates_Prime]
                       for date in sublist]

    dates_Prime_p2w = [date for sublist in
                       [pd.date_range(x + timedelta(8), x + timedelta(14)).tolist()
                        for x in dates_Prime]
                       for date in sublist]

    dates_Prime_p3w = [date for sublist in
                       [pd.date_range(x + timedelta(15), x + timedelta(21)).tolist()
                        for x in dates_Prime]
                       for date in sublist]
    df_holiday_binary.loc[dates_Prime, 'PD'] = 1
    df_holiday_binary.loc[dates_Prime_m1w, 'PD_m1w'] = 1
    df_holiday_binary.loc[dates_Prime_p1w, 'PD_p1w'] = 1
    df_holiday_binary.loc[dates_Prime_p2w, 'PD_p2w'] = 1
    df_holiday_binary.loc[dates_Prime_m1w, 'PD_m1w'] = 1
    df_holiday_binary.loc[dates_Prime_p1w, 'PD_p1w'] = 1
    
    if country in ['US','AU','CA']:
        dates_Alexa = query_holidays(country,channel,device_type,'Alexa\'\'s Birthday',start_year,end_year)
        dates_Alexa_m1w = [date for sublist in
                               [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                for x in dates_Alexa]
                               for date in sublist]
        dates_Alexa_p1w = [date for sublist in
                           [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                            for x in dates_Alexa]
                           for date in sublist]
        df_holiday_binary.loc[dates_Alexa, 'Alexa'] = 1
        df_holiday_binary.loc[dates_Alexa_m1w, 'Alexa_m1w'] = 1
        df_holiday_binary.loc[dates_Alexa_p1w, 'Alexa_p1w'] = 1
    if country == 'UK':
        dates_bank_summar = query_holidays(country,channel,device_type,'Summer Bank Holiday',start_year, end_year)
        dates_bank_spring = query_holidays(country, channel, device_type, 'Spring Bank Holiday', start_year, end_year)
        dates_bank_summar_m1w = [date for sublist in
                               [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                for x in dates_bank_summar]
                               for date in sublist]
        dates_bank_summar_p1w = [date for sublist in
                           [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                            for x in dates_bank_summar]
                           for date in sublist]
        dates_bank_spring_m1w = [date for sublist in
                               [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                for x in dates_bank_spring]
                               for date in sublist]
        dates_bank_spring_p1w = [date for sublist in
                           [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                            for x in dates_bank_spring]
                           for date in sublist]
        df_holiday_binary.loc[dates_bank_summar, 'summar_bank'] = 1
        df_holiday_binary.loc[dates_bank_spring, 'spring_bank'] = 1
        df_holiday_binary.loc[dates_bank_summar_m1w, 'summar_bank_m1w'] = 1
        df_holiday_binary.loc[dates_bank_spring_m1w, 'spring_bank_m1w'] = 1
        df_holiday_binary.loc[dates_bank_summar_p1w, 'summar_bank_p1w'] = 1
        df_holiday_binary.loc[dates_bank_spring_p1w, 'spring_bank_p1w'] = 1
        
    if country in ['US', 'CA', 'MX', 'UK', 'DE', 'FR', 'IT', 'ES', 'JP', 'BR', 'AU']:
        
        # remove easter for now 
        # dates_eastern = query_holidays(country, channel, device_type,'Easter Monday',start_year,end_year)
        # dates_eastern_m1w = [date for sublist in
        #                        [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
        #                         for x in dates_eastern]
        #                        for date in sublist]
        # dates_eastern_p1w = [date for sublist in
        #                    [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
        #                     for x in dates_eastern]
        #                    for date in sublist]
        # df_holiday_binary.loc[dates_eastern, 'eastern'] = 1
        # df_holiday_binary.loc[dates_eastern_m1w, 'easter_m1w'] = 1
        # df_holiday_binary.loc[dates_eastern_p1w, 'easter_p1w'] = 1
        
        dates_ThanksGiving = query_holidays(country, channel, device_type,'Thanksgiving',start_year,end_year)
        dates_ThanksGiving_m4w = [date for sublist in
                                  [pd.date_range(x - timedelta(28), x - timedelta(22)).tolist() for x in
                                   dates_ThanksGiving] for date in sublist]
    
        dates_ThanksGiving_m3w = [date for sublist in
                                  [pd.date_range(x - timedelta(21), x - timedelta(15)).tolist() for x in
                                   dates_ThanksGiving] for date in sublist]
    
        dates_ThanksGiving_m2w = [date for sublist in
                                  [pd.date_range(x - timedelta(14), x - timedelta(8)).tolist() for x in
                                   dates_ThanksGiving] for date in sublist]
    
        dates_ThanksGiving_m1w = [date for sublist in
                                  [pd.date_range(x - timedelta(7), x - timedelta(2)).tolist() for x in
                                   dates_ThanksGiving] for date in sublist]
    
        df_holiday_binary.loc[dates_ThanksGiving, 'TG'] = 1
        df_holiday_binary.loc[dates_ThanksGiving_m3w, 'TG_m3w'] = 1
        df_holiday_binary.loc[dates_ThanksGiving_m2w, 'TG_m2w'] = 1
        df_holiday_binary.loc[dates_ThanksGiving_m1w, 'TG_m1w'] = 1
    
        df_holiday_binary['TG_m1d'] = df_holiday_binary['TG'].shift(-1)
        df_holiday_binary['TG_p1d'] = df_holiday_binary['TG'].shift(1)
        df_holiday_binary['TG_p2d'] = df_holiday_binary['TG'].shift(2)
        df_holiday_binary['TG_p3d'] = df_holiday_binary['TG'].shift(3)
        df_holiday_binary['TG_p4d'] = df_holiday_binary['TG'].shift(4)
    
        dates_Xmas = query_holidays(country, channel, device_type, 'Christmas',start_year, end_year)
    
        dates_Xmas_m3w = [date for sublist in
                          [pd.date_range(x - timedelta(21), x - timedelta(15)).tolist()
                           for x in dates_Xmas]
                          for date in sublist]
    
        dates_Xmas_m2w = [date for sublist in
                          [pd.date_range(x - timedelta(14), x - timedelta(8)).tolist()
                           for x in dates_Xmas]
                          for date in sublist]
    
        dates_Xmas_m1w = [date for sublist in
                          [pd.date_range(x - timedelta(7), x - timedelta(2)).tolist()
                           for x in dates_Xmas]
                          for date in sublist]
    
        dates_Xmas_p1w = [date for sublist in
                          [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                           for x in dates_Xmas]
                          for date in sublist]
    
        dates_Xmas_p2w = [date for sublist in
                          [pd.date_range(x + timedelta(8), x + timedelta(14)).tolist()
                           for x in dates_Xmas]
                          for date in sublist]
    
        dates_Xmas_p3w = [date for sublist in
                          [pd.date_range(x + timedelta(15), x + timedelta(21)).tolist()
                           for x in dates_Xmas]
                          for date in sublist]
    
        dates_Xmas_p4w = [date for sublist in
                          [pd.date_range(x + timedelta(22), x + timedelta(28)).tolist()
                           for x in dates_Xmas]
                          for date in sublist]
    
        dates_Xmas_p5w = [date for sublist in
                          [pd.date_range(x + timedelta(29), x + timedelta(35)).tolist()
                           for x in dates_Xmas]
                          for date in sublist]
    
        df_holiday_binary.loc[dates_Xmas, 'CD'] = 1
        df_holiday_binary.loc[dates_Xmas_m3w, 'CD_m3w'] = 1
        df_holiday_binary.loc[dates_Xmas_m2w, 'CD_m2w'] = 1
        df_holiday_binary.loc[dates_Xmas_m1w, 'CD_m1w'] = 1
        df_holiday_binary['CDE'] = df_holiday_binary['CD'].shift(-1)
        
        if country in ['AU','CA','UK']:
            df_holiday_binary['BOX'] = df_holiday_binary['CD'].shift(1)
        
        dates_TG_CD = zip(sorted(dates_ThanksGiving), sorted(dates_Xmas))
    
        dates_ThanksGiving_p1w = [date for sublist in
                                  [pd.date_range(x[0] + timedelta(5),
                                                 x[1] - timedelta(22)).tolist()
                                   for x in dates_TG_CD]
                                  for date in sublist]
        df_holiday_binary.loc[dates_ThanksGiving_p1w, 'TG_p1w'] = 1
        if country == 'DE':
            fathers_day = [dt(2018, 5, 10), dt(2019, 5, 30), dt(2020,5,21), dt(2021, 5, 13)]
        else:
            fathers_day = query_holidays(country, channel, device_type, 'Father\'\'s Day',start_year, end_year)
        
        dates_father_m1w = [date for sublist in
                               [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                for x in fathers_day]
                               for date in sublist]
        dates_father_p1w = [date for sublist in
                           [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                            for x in fathers_day]
                           for date in sublist]
        dates_father_p2w = [date for sublist in
                           [pd.date_range(x + timedelta(8), x + timedelta(14)).tolist()
                            for x in fathers_day]
                           for date in sublist]
        df_holiday_binary.loc[fathers_day, 'FD'] = 1
        df_holiday_binary.loc[dates_father_m1w, 'FD_m1w'] = 1
        df_holiday_binary.loc[dates_father_p1w, 'FD_p1w'] = 1
        df_holiday_binary.loc[dates_father_p2w, 'FD_p2w'] = 1
        
        if country == 'DE':
            dates_whitmonday = query_holidays(country, channel, device_type,'Whit Monday',start_year,end_year)
            dates_whitmonday_m1w = [date for sublist in
                                   [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                    for x in dates_whitmonday]
                                   for date in sublist]
            dates_whitmonday_p1w = [date for sublist in
                               [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                                for x in dates_whitmonday]
                               for date in sublist]
            df_holiday_binary.loc[dates_whitmonday, 'whitmonday'] = 1
            df_holiday_binary.loc[dates_whitmonday_m1w, 'whitmonday_m1w'] = 1
            df_holiday_binary.loc[dates_whitmonday_p1w, 'whitmonday_p1w'] = 1
            
            dates_threeking = query_holidays(country, channel, device_type,'Three Kings Day',start_year,end_year)
            dates_threeking_m1w = [date for sublist in
                                   [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                    for x in dates_threeking]
                                   for date in sublist]
            dates_threeking_p1w = [date for sublist in
                               [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                                for x in dates_threeking]
                               for date in sublist]
            df_holiday_binary.loc[dates_threeking, 'threeking'] = 1
            df_holiday_binary.loc[dates_threeking_m1w, 'threeking_m1w'] = 1
            df_holiday_binary.loc[dates_threeking_p1w, 'threeking_p1w'] = 1
            
            dates_holiday = query_holidays(country, channel, device_type,'Labor Day',start_year,end_year)
            dates_holiday_m1w = [date for sublist in
                                   [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                    for x in dates_holiday]
                                   for date in sublist]
            dates_holiday_p1w = [date for sublist in
                               [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                                for x in dates_holiday]
                               for date in sublist]
            df_holiday_binary.loc[dates_holiday, 'laborday'] = 1
            df_holiday_binary.loc[dates_holiday_m1w, 'laborday_m1w'] = 1
            df_holiday_binary.loc[dates_holiday_p1w, 'laborday_p1w'] = 1
            
            dates_holiday = query_holidays(country, channel, device_type,'Labor Day',start_year, end_year)
            dates_holiday_m1w = [date for sublist in
                                   [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                    for x in dates_holiday]
                                   for date in sublist]
            dates_holiday_p1w = [date for sublist in
                               [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                                for x in dates_holiday]
                               for date in sublist]
            df_holiday_binary.loc[dates_holiday, 'laborday'] = 1
            df_holiday_binary.loc[dates_holiday_m1w, 'laborday_m1w'] = 1
            df_holiday_binary.loc[dates_holiday_p1w, 'laborday_p1w'] = 1
            
            dates_holiday = query_holidays(country, channel, device_type,'German Unity Day',start_year, end_year)
            dates_holiday_m1w = [date for sublist in
                                   [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                    for x in dates_holiday]
                                   for date in sublist]
            dates_holiday_p1w = [date for sublist in
                               [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                                for x in dates_holiday]
                               for date in sublist]
            df_holiday_binary.loc[dates_holiday, 'unityday'] = 1
            df_holiday_binary.loc[dates_holiday_m1w, 'unityday_m1w'] = 1
            df_holiday_binary.loc[dates_holiday_p1w, 'unityday_p1w'] = 1
            
            dates_holiday = query_holidays(country, channel, device_type,'Corpus Christi',start_year, end_year)
            dates_holiday_m1w = [date for sublist in
                                   [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                    for x in dates_holiday]
                                   for date in sublist]
            dates_holiday_p1w = [date for sublist in
                               [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                                for x in dates_holiday]
                               for date in sublist]
            df_holiday_binary.loc[dates_holiday, 'corpuschristi'] = 1
            df_holiday_binary.loc[dates_holiday_m1w, 'corpuschristi_m1w'] = 1
            df_holiday_binary.loc[dates_holiday_p1w, 'corpuschristi_p1w'] = 1
            
            dates_holiday = query_holidays(country, channel, device_type,'Assumption Day',start_year, end_year)
            dates_holiday_m1w = [date for sublist in
                                   [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                    for x in dates_holiday]
                                   for date in sublist]
            dates_holiday_p1w = [date for sublist in
                               [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                                for x in dates_holiday]
                               for date in sublist]
            df_holiday_binary.loc[dates_holiday, 'assumption'] = 1
            df_holiday_binary.loc[dates_holiday_m1w, 'assumption_m1w'] = 1
            df_holiday_binary.loc[dates_holiday_p1w, 'assumption_p1w'] = 1
            
            dates_holiday = query_holidays(country, channel, device_type,'Ascension Day',start_year, end_year)
            dates_holiday_m1w = [date for sublist in
                                   [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                    for x in dates_holiday]
                                   for date in sublist]
            dates_holiday_p1w = [date for sublist in
                               [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                                for x in dates_holiday]
                               for date in sublist]
            df_holiday_binary.loc[dates_holiday, 'accension'] = 1
            df_holiday_binary.loc[dates_holiday_m1w, 'accension_m1w'] = 1
            df_holiday_binary.loc[dates_holiday_p1w, 'accension_p1w'] = 1
            
    if (country == 'IN'):
        dates_Diwali = query_holidays(country, channel, device_type, 'Diwali',start_year, end_year)
 
        dates_Diwali_p1w = [date for sublist in
                            [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                             for x in dates_Diwali]
                            for date in sublist]
 
        df_holiday_binary.loc[dates_Diwali, 'Diwali'] = 1
        df_holiday_binary.loc[dates_Diwali_p1w, 'Diwali_p1w'] = 1
 
        dates_Dussehra = query_holidays(country, channel, device_type, 'Dussehra',start_year, end_year)
 
        dates_Dussehra_p1w = [date for sublist in
                              [pd.date_range(x + timedelta(1), x + timedelta(7)).tolist()
                               for x in dates_Dussehra]
                              for date in sublist]
 
        df_holiday_binary.loc[dates_Dussehra, 'Dussehra'] = 1
        df_holiday_binary.loc[dates_Dussehra_p1w, 'Dussehra_p1w'] = 1

        dates_Valentine = query_holidays(country, channel, device_type, 'Valentine\'\'s Day',start_year, end_year)
 
        dates_Valentine_m1w = [date for sublist in
                               [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                                for x in dates_Valentine]
                               for date in sublist]
 
        df_holiday_binary.loc[dates_Valentine, 'Valentine'] = 1
        df_holiday_binary.loc[dates_Valentine_m1w, 'Valentine_m1w'] = 1
 
        dates_Xmas = get_holiday_dates(settings,
                                       holiday_name='Christmas')
 
        dates_Xmas_m1w = [date for sublist in
                          [pd.date_range(x - timedelta(7), x - timedelta(1)).tolist()
                           for x in dates_Xmas]
                          for date in sublist]
 
        df_holiday_binary.loc[dates_Xmas, 'CD'] = 1
        df_holiday_binary.loc[dates_Xmas_m1w, 'CD_m1w'] = 1
    
    df_holiday_binary = df_holiday_binary.reset_index(drop=True)
    df_holiday_binary = df_holiday_binary.fillna(0)
    return df_holiday_binary

In [4]:
# helper functions to create seasonality, trend, lag features
def create_seasonality_features(period = 'yearly', n_range = None , dates = None): 

    delta_days = [(x - pd.Timestamp(x.year, 1, 1)).days for x in dates] # number of days since Janunary 1st
    if period == 'yearly': 
        P = 365.25
    elif period == 'monthly': 
        P = 30
    elif period == 'weekly': 
        P = 7
    sine = np.array([[np.sin(2.0*i*np.pi*n/P) for i in delta_days] for n in n_range]).T
    col_sine_names = ['_'.join(['seasonal', period , 'fourier', 'sine', str(n)]) for n in n_range]
    cosine = np.array([[np.cos(2.0*i*np.pi*n/P) for i in delta_days] for n in n_range]).T
    col_cosine_names = ['_'.join(['seasonal', period , 'fourier', 'cosine', str(n)]) for n in n_range]
    df_sine = pd.DataFrame(sine, columns = col_sine_names)
    df_cosine = pd.DataFrame(cosine, columns = col_cosine_names)
    df_sine['date'] = dates
    df_cosine['date'] = dates
    
    return {'sine':df_sine, 'cosine':df_cosine}


def create_trend_features(df, period = 'semi', trend_start_year = 2017):
    """
        create trend features based on period
    """ 

    dates = df['date']

    if period == 'annual': 
        n_days = int(365)
    elif period == 'semi-annual': 
        n_days = int(365/2)
    elif period == 'quarterly': 
        n_days = int(365/4) + 1
    elif period == 'monthly': 
        n_days = 30
    else: 
        raise('Choose period from [annual, semi-annual, quarterly, monthly]')

    trend_start_date = pd.Timestamp(trend_start_year,1,1) # trend start date 
    current_date = pd.to_datetime('today').date() 
    
    trend_referenced_dates = pd.date_range(trend_start_date, current_date, freq = '{}D'.format(n_days)) # generate referenced dates based on freq
    delta_days = np.array([[(x - trend_referenced_date).days for x in dates] for trend_referenced_date in trend_referenced_dates]) 
    df_trend = pd.DataFrame(delta_days.T, columns = ['_'.join([period, 'trend_since', str(x.year), str(x.month)]) for x in trend_referenced_dates])
    df_trend[df_trend < 0] = 0 # set all negative values to 0 
    df_trend = np.log(df_trend + 1.0) # log-transform
    df_trend['date'] = dates
    df_trend['asin'] = df.asin

    return df_trend

def generate_lag_lead_features(df, lags):
    
    # helper function
    def lag_by_group(key, value_df, lag = 1):
        df = value_df.assign(group = key) # this pandas method returns a copy of the df, with group columns assigned the key value
        return (df.sort_values(by=["date"], ascending=True)
            .set_index(["date"])
            .shift(lag))
    
    temp = df
    for lag in lags: 
        grouped_df = df[['asp', 'vlt', 'asin', 'date']].groupby(["asin"])
        dflist = [lag_by_group(g, grouped_df.get_group(g), lag) for g in grouped_df.groups.keys()]
        result_lags = pd.concat(dflist, axis=0).reset_index()
        if lag > 0: 
            columns = ['date', '_'.join(['asp','lag', str(lag)]), '_'.join(['vlt','lag', str(lag)]) , 'asin']
        if lag < 0: 
            columns = ['date', '_'.join(['asp','lead', str(-lag)]), '_'.join(['vlt','lead', str(-lag)]) , 'asin']
        result_lags = result_lags.drop('group', axis = 1)
        result_lags.columns = columns
        temp = pd.merge(temp, result_lags, on = ['date', 'asin'], how = 'left')

    temp = temp.drop(['asp', 'vlt'], axis = 1)

    return temp

In [5]:
def pre_process(country, channel, device, start_date, end_date):
    

    df_vlt = query_vlt(country, channel, device, start_date, end_date)
    df_asp = query_asp(country, channel, device, start_date, end_date)
    df_actuals = query_actuals(country, channel, device, pd.to_datetime(start_date), pd.to_datetime(end_date))
    
    df_age = query_asin_end(country, channel, device)
    df_age['life_value_street'] = (df_age['asin_end_date'] - df_age['asin_start_date']).dt.days
    df_age['log_age'] = np.log(df_age['life_value_street'])
    df_all = pd.merge(df_asp, df_age, on = ['asin', 'program'], how = 'left')
    df_all['age'] = np.log((df_all['date'] - df_all['asin_start_date']).dt.days + 1.0)

    df_asin_attr = query_asin_attr(country, channel, device, start_date, end_date)
    df_asin_attr = df_asin_attr.drop_duplicates(subset = 'asin') # drop duplicates
    df_all = pd.merge(df_all, df_asin_attr[['asin', 'storage', 'bundle', 'dtcp', 'ket_color', 'msrp', 'color_name', 'sub_category']], on = ['asin'], how = 'left')
    df_all = pd.merge(df_all, df_vlt, on = ['date', 'asin'], how = 'left')
    # process nan's 
    df_all.loc[df_all.alp.isna(), 'alp'] = df_all.loc[df_all.alp.isna(), 'msrp'] # replace alp nans by msrp
    df_all.loc[df_all.asp.isna(), 'asp'] = df_all.loc[df_all.asp.isna(), 'alp'] # replace asp nans by alp 
    df_all['asp'] = np.log(df_all['asp']) # take log of asp and call asp for simplicity
    df_all['discount'] = (df_all['alp'] - df_all['asp'])/df_all['alp']
    df_all.loc[df_all.discount.isna(), 'discount'] = 0.0  # set discount nan's to 0
    df_all.loc[df_all.vlt > 365, 'vlt'] = 365
    df_all['vlt'] = np.log(df_all['vlt'] + 1.0)
    df_all.loc[df_all.vlt.isna(), 'vlt'] = 0.0
    df_all = pd.merge(df_all, df_actuals[['date', 'asin', 'actual']], on = ['date', 'asin'], how = 'left')
    
    # generate lead and lag variables
    df_lags = generate_lag_lead_features(df_all[['asp', 'vlt', 'asin', 'date']], lags = range(1,7)) # 7 lags
    df_leads = generate_lag_lead_features(df_all[['asp', 'vlt', 'asin', 'date']], lags = range(-7,0)) # 7 leads
    temp = pd.merge(df_lags, df_leads, on = ['asin', 'date'])

    dict_seasonality = create_seasonality_features(period = 'yearly', n_range = range(1,21) , dates = pd.to_datetime(df_asp.date.unique()))
    temp = pd.merge(temp, dict_seasonality['sine'], on = 'date', how = 'left')
    temp = pd.merge(temp, dict_seasonality['cosine'], on = 'date', how = 'left')

    # create annual trend
    for period in ['annual']: 
        df_trend = create_trend_features(period = period, df = temp[['date','asin']])
        temp = pd.merge(temp, df_trend, on = ['date', 'asin'])

    df_all = pd.merge(df_all, temp, on = ['date', 'asin'], how = 'left')

    return df_all

In [6]:
def convert_one_hot_encoding(df, drop_first = False):
    unique_values = df.apply(lambda x: len(x.unique())).values
    cat_vars = df.columns[(unique_values < 40) & (unique_values > 2)]
    for var in cat_vars:
        if var in df.columns:
            if var not in ['asin', 'covid', 'log_age', 'vlt', 'Customer-Facing Name', 'log_asp', 'log_alp', 'program']:
                df = df.join(pd.get_dummies(df[var],prefix = var, drop_first = drop_first))
                if var != 'program':
                    df = df.drop(var, axis = 1)
    return df

In [7]:
def generate_df_input(country, channel, device, start_date, end_date):

    df = pre_process(country, channel, device, start_date, end_date)
    df_holidays = generate_holidays(country, channel, device, start_date, end_date)
    df_category = pd.read_csv('program_category.csv') # will fix this for Eider note book 
    df = pd.merge(df, df_category, on = 'program', how = 'left')
    df_holidays = generate_holidays(country, channel, device, start_date, end_date)
    df = pd.merge(df, df_holidays, on = 'date', how = 'left')
    names = df['Customer-Facing Name']
    df['is_dot'] =  np.array(['Dot' in x for x in names], dtype = 'int')
    df['is_show'] = np.array(['Show' in x for x in names], dtype = 'int')
    df['is_plus'] = np.array(['Plus' in x for x in names], dtype = 'int')
    df['covid'] = df.date >= '2020-03-15'
    df['weekday'] = np.array(df['date'].apply(lambda x: x.weekday()))
    df['month'] = np.array(df['date'].apply(lambda x: x.month))

    return df

# df_temp = convert_one_hot_encoding(df)
# df_temp = df_temp[(~df_temp.actual.isna()) & (df_temp.date <= END_DATE)]
# return df


In [8]:
def run_backcast(df, BACKCAST_START_DATE, BACKCAST_END_DATE, horizon = 24):


    drop_vars = ['channel', 'asin', 'log_actual', 'baseline', 'actual_last', 'msrp', 
                'nrows', 'country', 'channel', 'vdps', 'wireleconvert_one_hot_encodingsqrt_actual',  'age', 
            'Customer-Facing Name', 'qty', 'ket_color', 'life_value_street', 'asin_end_date', 'Category', 'bundle', 'dtcp', 'device_type', 'alp', 'asin_start_date']
    for var in drop_vars: 
        if var in df.columns: 
            df = df.drop(var, axis = 1)

    
    df_temp = convert_one_hot_encoding(df)
    df_temp = df_temp[(~df_temp.actual.isna()) & (df_temp.date <= END_DATE)]

    remove_vars = df_temp.columns[['vlt_lead' in x for x in df_temp.columns]]
    df_temp = df_temp.drop(remove_vars, 1)


#     remove_vars = df_temp.columns[['asp_lag' in x for x in df_temp.columns]]
#     df_temp = df_temp.drop(remove_vars, 1)

#     remove_vars = df_temp.columns[['annual_trend' in x for x in df_temp.columns]]
#     df_temp = df_temp.drop(remove_vars, 1)

#     remove_vars = df_temp.columns[['quarterly_trend' in x for x in df_temp.columns]]
#     df_temp = df_temp.drop(remove_vars, 1)


    BACKCAST_PERIODS = pd.date_range(start = BACKCAST_START_DATE, end = BACKCAST_END_DATE , freq = 'W-MON')

    ridge_model = CCARD_MODEL(feature_selection = True, hyperparameter_tuning = False,  random_state = 0, model_name = 'ridge', transformation = 'log')
    df_horizon, df_result_all = ridge_model.back_cast(df_temp, BACKCAST_PERIODS, BACKCAST_END_DATE, horizon, retrain = True, metric_clip = True)

    return df_horizon.sort_values('program')


In [9]:
def generate_forecast(df,  training_start_date, training_end_date, forecast_end_date):
    
    forecast_end_date = min(pd.to_datetime(training_end_date) + timedelta(30*7), pd.to_datetime(forecast_end_date))

    drop_vars = ['channel', 'asin', 'log_actual', 'baseline', 'actual_last', 'msrp', 
                'nrows',  'vdps', 'wireless_type', 'sqrt_actual',  'age', 
            'Customer-Facing Name', 'qty', 'ket_color', 'life_value_street', 'asin_end_date', 'Category', 'bundle', 'dtcp',  'alp', 'asin_start_date']

    df_final_forecast = df.query('date >= @training_end_date & date < @forecast_end_date')[['date','country','channel','device_type','program','asin','actual']]


    for var in drop_vars: 
        if var in df.columns: 
            df = df.drop(var, axis = 1)

    df = convert_one_hot_encoding(df)
    df_train = df.query('date < @training_end_date')

    df_train = df_train[~df_train.actual.isna()]
    ridge_model = CCARD_MODEL(feature_selection = True, hyperparameter_tuning = True,  random_state = 0, model_name = 'ridge', transformation = 'log')
    ridge_model.fit(df_train.drop('actual', axis = 1), np.log(df_train['actual'] + 0.0001))

    df_pred = df.query('date >= @training_end_date & date < @forecast_end_date')
    y_pred = ridge_model.predict(df_pred[ridge_model.selected_cols])
    df_final_forecast['vdp_plus_units'] = y_pred

    return df_final_forecast


In [10]:
%%time
from LR_prod import *
COUNTRY = 'US'  
CHANNEL='ONLINE'
DEVICE = 'AuCC'
START_DATE= '2016-05-01'
END_DATE='2020-12-31'
BACKCAST_START_DATE = '2019-11-04'
BACKCAST_END_DATE = '2020-05-25'
COVID_START = '2020-03-15' # assume covid start at the beginning of march

training_end_date = '2020-06-24'
forecast_end_date = '2020-12-31'

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 12.9 µs


In [11]:
df = generate_df_input(COUNTRY, CHANNEL, DEVICE, START_DATE, END_DATE)

In [20]:
backcast_result = run_backcast(df, BACKCAST_START_DATE, BACKCAST_END_DATE, horizon = 24)
print(backcast_result)

Fold 1
Preparing training and test sets ...
Training model up to 2019-11-04 and forecasting up to 2020-04-20
Running feature selection ...
Dropping constant vars: ['covid']
The best param(s) are {'alpha': 0.001}
Fold 2
Preparing training and test sets ...
Training model up to 2019-11-11 and forecasting up to 2020-04-27
Running feature selection ...
Dropping constant vars: ['covid']
The best param(s) are {'alpha': 0.001}
Fold 3
Preparing training and test sets ...
Training model up to 2019-11-18 and forecasting up to 2020-05-04
Running feature selection ...
Dropping constant vars: ['covid']
The best param(s) are {'alpha': 0.001}
Fold 4
Preparing training and test sets ...
Training model up to 2019-11-25 and forecasting up to 2020-05-11
Running feature selection ...
Dropping constant vars: ['covid']
The best param(s) are {'alpha': 0.001}
Fold 5
Preparing training and test sets ...
Training model up to 2019-12-02 and forecasting up to 2020-05-18
Running feature selection ...
Dropping cons

In [ ]:
# generate future forecasts
df_final_forecast = generate_forecast(df, START_DATE, training_end_date, forecast_end_date)
print(df_final_forecast[df_final_forecast['actual'] > 0.0])

running feature selection ...
Running hyperparameter tuning ...


In [18]:
# check in sample Covid period
df_final_forecast = generate_forecast(df, START_DATE, '2020-03-15', '2020-07-18')
df_final_forecast['actual'].sum(), df_final_forecast['vdp_plus_units'].sum()

running feature selection ...
Running hyperparameter tuning ...


(3280839.0, 2391755.8152228035)

,date,country,channel,device_type,program,asin,actual,vdp_plus_units
2877,2020-06-24,US,ONLINE,AuCC,Crumpet,B07PX3SCFN,159.0,121.569439
2878,2020-06-25,US,ONLINE,AuCC,Crumpet,B07PX3SCFN,155.0,122.868041
2879,2020-06-26,US,ONLINE,AuCC,Crumpet,B07PX3SCFN,146.0,120.640974
2880,2020-06-27,US,ONLINE,AuCC,Crumpet,B07PX3SCFN,179.0,118.115260
2881,2020-06-28,US,ONLINE,AuCC,Crumpet,B07PX3SCFN,192.0,123.423165
...,...,...,...,...,...,...,...,...
31762,2020-07-14,US,ONLINE,AuCC,Lidar,B07CT3W44K,98.0,41.749391
31763,2020-07-15,US,ONLINE,AuCC,Lidar,B07CT3W44K,123.0,39.143750
31764,2020-07-16,US,ONLINE,AuCC,Lidar,B07CT3W44K,305.0,36.148382
31765,2020-07-17,US,ONLINE,AuCC,Lidar,B07CT3W44K,171.0,23.009991
